Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

Create a small validation set.

Utility functions to map characters to vocabulary IDs and back.

Function to generate a training batch for the LSTM model.

Simple LSTM Model.

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [4]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [5]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [6]:
vocabulary_size = 729

def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
#  print('predictions is %s' % predictions)
#  print('labels is %s' % labels)
#  print('label shape is %s' % str(labels.shape))
#  print('labels[0] shape is %s' % labels[0])
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [7]:
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0

def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '


def chars2eid(chars):
  return char2id(chars[0]) * 27 + char2id(chars[1])

def eid2chars(eid):
  if eid > 0:
    return id2char(eid // 27) + id2char(eid % 27)
  else:
    return '  '

In [18]:
batch_size = 64
num_unrollings = 10
#vocabulary_size = 128   # since we are now projecting our 729 to 128

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size / batch_size
        self._cursor = [ offset * segment for offset in xrange(batch_size)]
        self._last_batch, self._last_label = self._next_batch()
#        self._last_label =  self._next_batch()
  
        
    def _next_batch(self):
  
        batch = np.zeros(shape=(self._batch_size), dtype = np.int32)
        label = np.zeros(shape=(self._batch_size,vocabulary_size), dtype = np.int64)
        for b in range(self._batch_size):
            
          c = self._cursor[b]
#          print("batch cursor is %s" % c)
#          batch[b] = chars2eid(self._text[c:c+2])
          bigram = chars2eid(self._text[c:c+2])
          batch[b] = bigram
          label[b, bigram] = 1.0
          self._cursor[b] = (self._cursor[b] + 2) % self._text_size
        return batch, label
  

    def next(self):
        batches = []
        labels = []
        batches = [self._last_batch]
        labels = [self._last_label]
        for step in range(self._num_unrollings):
          batch, label =  self._next_batch()
          batches.append(batch)
          labels.append(label)
        
        self._last_batch = batches[-1]
        self._last_label = labels[-1]
        return (batches, labels)
    
def batches2string(batches):
    s = [''] * batches[0].shape[0]

    for b in batches:
   
        s = [''.join(x) for x in zip(s, characters(b))]

    return s

def characters(probabilities):
 
    return [eid2chars(c) for c in np.argmax(probabilities, 1)]
 
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
print("train_batches._last_batch is %s" % str(train_batches._last_batch))
print("train_batches._last_label is %s" % str(train_batches._last_label))

#valid_batches = BatchGenerator(valid_text, 1, 1)
#print("valid_batches._last_batch is %s" % str(valid_batches._last_batch))


print('#' * 100)

#test2,test_label2 = train_batches.next()

test3,test_label3 = valid_batches.next()
print("valid batch[0] is %s" % test3[0])
print("valid batch[1] is %s" % test3[1])
print("valid batch[0] type is %s" % type(test3[0]))
print("valid label is %s" % test_label3[0])
print("valid label is %s" % test_label3[1])


train_batches._last_batch is [419 629 336   1 352 221 675  52 549 360 383 221 135 137 405 680 423  27
  20  45 263   1 549 133 162  47 135 149  40 508 258 405  27 197 257 501
  82  46   4 366 540 567 135 405 411 513 309 153 640 155 548 155  19 262
 549 108 548 149 167 129 558  47  43 522]
train_batches._last_label is [[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
####################################################################################################
valid batch[0] is [379]
valid batch[1] is [545]
valid batch[0] type is <type 'numpy.ndarray'>
valid label is [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---